In [103]:
import ifcb
import csv
from urllib.request import urlopen
import numpy as np
import os
import imageio
from pathlib import Path

In [104]:
def create_folders(output_path,classes):
    #os.mkdir(output_path)
    for folder_name in classes:
        path = output_path + '/' + folder_name
        try:
            #os.mkdir(path)
            Path(path).mkdir(parents=True, exist_ok=True)
        except OSError:
            print ("Creation of the directory %s failed" % path)

In [111]:
def classify_images_in_folders(base_url,output_path,pid,threshold):
    with urlopen(base_url + pid + '_class_scores.csv') as f :
        reader = csv.reader(f.read().decode('utf-8').split('\n'),delimiter=',')
        classes = next(reader)[1:]
        classes.append('Unclassified')
        results = {k:0 for k in classes}
        create_folders(output_path,classes)
        roi_numbers = []
        best_classes = []
        for row in reader:
            if len(row) > 0:
                test = row
                roi_number = int(row[0].split("_")[-1])
                roi_numbers.append(roi_number)
                scores = [float(score) for score in row[1:]]
                max_score = np.amax(scores)
                if max_score > threshold:
                    best_class = np.where(scores == max_score)[0][0]
                else:
                    best_class = classes.index('Unclassified')
                best_classes.append(best_class)
         # Retrieving images
        with ifcb.open_url(base_url + pid) as sample_bin:
            print('{} has {} image(s)'.format(sample_bin.lid, len(sample_bin.images)))
            for roi_number in sample_bin.images:
                pos = roi_numbers.index(roi_number)
                best_class = best_classes[pos]
                class_name = classes[best_class]
                results[class_name] = results[class_name] + 1 # Counting images
                img_path = output_path + '/' + class_name + '/' +  pid + '_' + str(roi_number) + '.png'
                imageio.imwrite(img_path, sample_bin.images[roi_number])
        return results

In [117]:
#pids_file = "D20200724_IFCB129_bins.csv"
pids_file = "D20200608_IFCB010_bins.csv"
base_url = 'https://ifcb-data.whoi.edu/mvco/'
output_path = 'run-output/SAMS-training-data'
threshold = 0.5

In [118]:
with open(pids_file) as f:
    reader = csv.reader(f)
    for row in reader:
        pid = row[0]
        results = classify_images_in_folders(base_url,output_path,pid,threshold)
        print(results['Dactyliosolen_blavyanus'])

D20200608T203418_IFCB010 has 5174 image(s)
11
D20200608T201224_IFCB010 has 5308 image(s)
24
D20200608T195031_IFCB010 has 5143 image(s)
16
D20200608T192838_IFCB010 has 5304 image(s)
12
D20200608T190645_IFCB010 has 5072 image(s)
15
D20200608T184452_IFCB010 has 4917 image(s)
11
D20200608T182259_IFCB010 has 4969 image(s)
21
D20200608T180106_IFCB010 has 5181 image(s)
28
D20200608T173911_IFCB010 has 5364 image(s)
29
D20200608T171718_IFCB010 has 5818 image(s)
28
D20200608T165524_IFCB010 has 5584 image(s)
19
D20200608T163331_IFCB010 has 6608 image(s)
17
D20200608T161138_IFCB010 has 5818 image(s)
21
D20200608T154945_IFCB010 has 5659 image(s)
17
D20200608T145909_IFCB010 has 5524 image(s)
30
D20200608T143715_IFCB010 has 5819 image(s)
26
D20200608T141522_IFCB010 has 6040 image(s)
47
D20200608T135329_IFCB010 has 6121 image(s)
41
D20200608T133136_IFCB010 has 5839 image(s)
28
D20200608T130943_IFCB010 has 5840 image(s)
29
D20200608T124750_IFCB010 has 6066 image(s)
44
D20200608T122557_IFCB010 has 6050 

In [116]:
results

{'Acanthoica_quattrospina': 0,
 'Alexandrium_catenella': 2,
 'Amphidinium': 0,
 'Amylax': 0,
 'Apedinella': 0,
 'Asterionellopsis_glacialis': 0,
 'Bacillaria': 0,
 'Bacillariophyceae': 57,
 'Balanion': 2,
 'Biddulphia': 0,
 'Calciopappus': 0,
 'Calciosolenia_brasiliensis': 0,
 'Cerataulina_pelagica': 9,
 'Ceratium': 0,
 'Ceratium_furca': 0,
 'Ceratium_fusus': 0,
 'Ceratium_lineatum': 0,
 'Chaetoceros': 7,
 'Chaetoceros_TAG_external_pennate': 0,
 'Chaetoceros_curvusetus': 0,
 'Chaetoceros_danicus': 0,
 'Chaetoceros_didymus': 0,
 'Chaetoceros_didymus_TAG_external_flagellate': 0,
 'Chaetoceros_similis': 0,
 'Chaetoceros_subtilis': 0,
 'Chaetoceros_throndsenii': 0,
 'Chroomonas': 1,
 'Chrysochromulina': 0,
 'Chrysochromulina_lanceolata': 0,
 'Corethron_hystrix': 0,
 'Corymbellus': 0,
 'Coscinodiscus': 0,
 'Cylindrotheca': 17,
 'Dactyliosolen_blavyanus': 1,
 'Dactyliosolen_fragilissimus': 2,
 'Delphineis': 1,
 'Dictyocha': 0,
 'Dictyocysta': 0,
 'Didinium': 0,
 'Dinobryon': 0,
 'Dinophyceae